# DM (device manufacturer) connects to music (device) and assign them ownership - DM

## 1. Create Invitation from agent

In [1]:
%autoawait
import time
import asyncio
from aries_basic_controller.aries_controller import AriesAgentController
    
WEBHOOK_HOST = "0.0.0.0"
WEBHOOK_PORT = 8022
WEBHOOK_BASE = ""
ADMIN_URL = "http://alice-agent:8021"

# WARNING: You should use environment variables for this
# TODO: Make env variables accessible through juypter notebooks
API_KEY = "alice_api_123456789"

# Based on the aca-py agent you wish to control
agent_controller = AriesAgentController(admin_url=ADMIN_URL, api_key=API_KEY)
agent_controller.init_webhook_server(webhook_host=WEBHOOK_HOST, 
                                     webhook_port=WEBHOOK_PORT,
                                     webhook_base=WEBHOOK_BASE)

loop = asyncio.get_event_loop()
loop.create_task(agent_controller.listen_webhooks())

def connection_handler(payload):
    print("Connection Handler Called")
    connection_id = payload["connection_id"]
    state = payload["state"]
    print(f"Connection {connection_id} in State {state}")
    
connection_listener = {
    "handler": connection_handler,
    "topic": "connections"
}

agent_controller.register_listeners([connection_listener], defaults=True)


IPython autoawait is `on`, and set to use `asyncio`
Subscribing too: connections


## 2. Create invitation

In [2]:
# Create Invitation
invite = await agent_controller.connections.create_invitation()
connection_id = invite["connection_id"]
invite_message = invite['invitation']
print("Connection ID", connection_id)
print("Invitation")
print(invite_message)

Connection Handler Called
Connection 031f3c7c-1b71-49b8-aded-1fd8bea8f3d8 in State invitation
Connection ID 031f3c7c-1b71-49b8-aded-1fd8bea8f3d8
Invitation
{'@type': 'did:sov:BzCbsNYhMrjHiqZDTUASHg;spec/connections/1.0/invitation', '@id': '288f4adf-6895-4104-8808-2bbed5ccd8ba', 'label': 'Alice', 'recipientKeys': ['8fzHn56Ws5XGrbnFgFC7cvNxAJaQ5H6twaRWGA8TKphM'], 'serviceEndpoint': 'http://192.168.65.3:8020'}
Connection Handler Called
Connection 031f3c7c-1b71-49b8-aded-1fd8bea8f3d8 in State request


## 4. Accept Request - Recipient first!!!!!

In [3]:
# Accept Request for Invite created
connection = await agent_controller.connections.accept_request(connection_id)
print("ACCEPT REQUEST")
print(connection)
print("state", connection["state"])

Connection Handler Called
Connection 031f3c7c-1b71-49b8-aded-1fd8bea8f3d8 in State response
ACCEPT REQUEST
{'routing_state': 'none', 'connection_id': '031f3c7c-1b71-49b8-aded-1fd8bea8f3d8', 'state': 'response', 'their_role': 'invitee', 'invitation_key': '8fzHn56Ws5XGrbnFgFC7cvNxAJaQ5H6twaRWGA8TKphM', 'my_did': 'QRT3h2UtVYUFwkjVEJjK5U', 'their_label': 'Bob', 'rfc23_state': 'response-sent', 'their_did': 'VjHmiqvFUDKtfjzadgFhjk', 'invitation_mode': 'once', 'accept': 'manual', 'created_at': '2023-01-03 09:28:01.122823Z', 'updated_at': '2023-01-03 09:29:11.912500Z'}
state response


In [4]:
trust_ping = await agent_controller.messaging.trust_ping(connection_id, "hello")
print("Trust Ping", trust_ping)

connection = await agent_controller.connections.get_connection(connection_id)
print(connection)
print("Is Active?", connection["state"])
connection_id = connection["connection_id"]

Trust Ping {'thread_id': 'b1d47850-f2c9-4137-a29a-ef87a346ac3f'}
{'routing_state': 'none', 'connection_id': '031f3c7c-1b71-49b8-aded-1fd8bea8f3d8', 'state': 'response', 'their_role': 'invitee', 'invitation_key': '8fzHn56Ws5XGrbnFgFC7cvNxAJaQ5H6twaRWGA8TKphM', 'my_did': 'QRT3h2UtVYUFwkjVEJjK5U', 'their_label': 'Bob', 'rfc23_state': 'response-sent', 'their_did': 'VjHmiqvFUDKtfjzadgFhjk', 'invitation_mode': 'once', 'accept': 'manual', 'created_at': '2023-01-03 09:28:01.122823Z', 'updated_at': '2023-01-03 09:29:11.912500Z'}
Is Active? response
Connection Handler Called
Connection 031f3c7c-1b71-49b8-aded-1fd8bea8f3d8 in State active


# Give MUSIC the VC of ownership and origin - DM

## 1. Write Ownership schema

In [5]:
# Define you schema name - must be unique on the ledger
schema_name = "music_schema"
# Can version the schema if you wish to update it
schema_version = "0.0.1"
# Define any list of attributes you wish to include in your schema
attributes = ["origin", "name", "time", "version", "author", "owner"]

# Write schema to ledger
response = await agent_controller.schema.write_schema(schema_name, attributes, schema_version)
schema_id = response["schema_id"]
print(schema_id)

# Write cred. def.
response = await agent_controller.definitions.write_cred_def(schema_id)
cred_def_id = response["credential_definition_id"]
print(cred_def_id)


# # Update schema
# attributes = ["author", "name", "time", "is_original", "type"]
# schema_version = "0.0.2"
# response = await agent_controller.schema.write_schema(schema_name, attributes, schema_version)
# updated_schema_id = response["schema_id"]
# print(updated_schema_id)

# # Write another Credential Definition for the Updated Schema
# response = await agent_controller.definitions.write_cred_def(updated_schema_id, support_revocation=True)
# updated_cred_def_id = response["credential_definition_id"]
# print(updated_cred_def_id)

PQRXDxdGqQGSZ8z69p4xZP:2:music_schema:0.0.1
PQRXDxdGqQGSZ8z69p4xZP:3:CL:12:default


## 2. Write VC

In [6]:
nym = await agent_controller.wallet.get_public_did()
print(nym["result"]["did"])
created_schema = await agent_controller.schema.get_created_schema(schema_issuer_did='PQRXDxdGqQGSZ8z69p4xZP')
print(created_schema)
schema_id = created_schema["schema_ids"][-1]

PQRXDxdGqQGSZ8z69p4xZP
{'schema_ids': ['PQRXDxdGqQGSZ8z69p4xZP:2:music_schema:0.0.1']}


In [7]:
definitions = await agent_controller.definitions.search_created(schema_id=schema_id)
print(definitions["credential_definition_ids"])
cred_def_id = definitions["credential_definition_ids"][-1]

['PQRXDxdGqQGSZ8z69p4xZP:3:CL:12:default']


In [8]:
response = await agent_controller.connections.get_connections()
results = response['results']
print(len(results))
# print("Results : ", results)
if len(results) > 0:
    connection = response['results']
    for con in connection:
        print("Connection :", con)
        if con['state'] == 'active':       
            connection_id = con["connection_id"]
            print("Active Connection ID : ", connection_id)
        else: print("Inactive Connection ID: ", con["connection_id"], "with state: ", con["state"])
else:
    print("You must create a connection")

1
Connection : {'routing_state': 'none', 'connection_id': '031f3c7c-1b71-49b8-aded-1fd8bea8f3d8', 'state': 'active', 'their_role': 'invitee', 'invitation_key': '8fzHn56Ws5XGrbnFgFC7cvNxAJaQ5H6twaRWGA8TKphM', 'my_did': 'QRT3h2UtVYUFwkjVEJjK5U', 'their_label': 'Bob', 'rfc23_state': 'completed', 'their_did': 'VjHmiqvFUDKtfjzadgFhjk', 'invitation_mode': 'once', 'accept': 'manual', 'created_at': '2023-01-03 09:28:01.122823Z', 'updated_at': '2023-01-03 09:29:15.132956Z'}
Active Connection ID :  031f3c7c-1b71-49b8-aded-1fd8bea8f3d8


In [9]:
import datetime
time = datetime.datetime.now().strftime("%m-%d-%Y, %H:%M:%S")
credential_attributes = [
    {"name": "author", "value": nym["result"]["did"]},
    {"name": "name", "value": "balabababa music"},
    {"name": "time", "value": time},
    {"name": "origin", "value": nym["result"]["did"]},
    {"name": "owner", "value": nym["result"]["did"]},
    {"name": "version", "value": "v1"}
]
print(credential_attributes)


# send credential
record = await agent_controller.issuer.send_credential(connection_id, schema_id, cred_def_id, credential_attributes, auto_remove=False, trace=True)
record_id = record['credential_exchange_id']
state = record['state']
role = record['role']
print(f"Credential exchange {record_id}, role: {role}, state: {state}")

[{'name': 'author', 'value': 'PQRXDxdGqQGSZ8z69p4xZP'}, {'name': 'name', 'value': 'balabababa music'}, {'name': 'time', 'value': '01-03-2023, 09:31:44'}, {'name': 'origin', 'value': 'PQRXDxdGqQGSZ8z69p4xZP'}, {'name': 'owner', 'value': 'PQRXDxdGqQGSZ8z69p4xZP'}, {'name': 'version', 'value': 'v1'}]
Credential exchange a30a96db-cc26-44c5-a847-e52d24ba1780, role: issuer, state: offer_sent


## 4. Wait for credential exchange

In [10]:
response = await agent_controller.issuer.get_records()
print(response['results'])
cred_record = await agent_controller.issuer.get_record_by_id(record_id)
state = cred_record['state']
role = cred_record['role']
print(f"Credential exchange {record_id}, role: {role}, state: {state}")

[{'credential_offer_dict': {'@type': 'did:sov:BzCbsNYhMrjHiqZDTUASHg;spec/issue-credential/1.0/offer-credential', '@id': '9c87fce3-a799-41f9-a3e7-4e1b5d1b01ec', '~thread': {}, '~trace': {'target': 'log', 'full_thread': True, 'trace_reports': []}, 'comment': 'create automated credential exchange', 'offers~attach': [{'@id': 'libindy-cred-offer-0', 'mime-type': 'application/json', 'data': {'base64': 'eyJzY2hlbWFfaWQiOiAiUFFSWER4ZEdxUUdTWjh6NjlwNHhaUDoyOm11c2ljX3NjaGVtYTowLjAuMSIsICJjcmVkX2RlZl9pZCI6ICJQUVJYRHhkR3FRR1NaOHo2OXA0eFpQOjM6Q0w6MTI6ZGVmYXVsdCIsICJrZXlfY29ycmVjdG5lc3NfcHJvb2YiOiB7ImMiOiAiNjE4MzgxNjA3MTg5ODI4NjM0NjQ0MjYxNTYxMjg2ODM5MDIwMTkxMDIyMjI3ODcwMDU2Mzk3NTU3NDU2NDAyMjEyOTM3MDQyMzY5NDgiLCAieHpfY2FwIjogIjczOTI4NzY0MTI1MzE5MjQ4MDg3MjQyNjQ5OTE2NDIxODU1NDQ2ODI0MzA0NzkyNzM1MTA2MjYzMzU2MzQzMzg5MjYzMjYyNTcxNDQyNTc4OTkyMzg2MDE2Nzc1Mjc1NDM4MDAzMTIwMDQ2OTQ3NzA1NTgzMzY4ODgzMzg1MDcxOTM5NzM5OTE4NDc5ODM4Njk4NTEzODQxNzMwOTg2MDI2ODk0MjI5Njc3OTk2MTA2NTc1MTk0NDAzNDY1NzkwOTIyNDk1OTc2ODc5Njc0MjY

In [24]:
## Remove records
response = await agent_controller.issuer.remove_record(record_id)
print(response)

{}


In [11]:
response = await agent_controller.terminate()
print(response)

None
